# Generate reference data

Make reference data for `carsus/io/output/base.py` module.

In [ ]:
import pandas as pd
from pandas.testing import assert_frame_equal, assert_series_equal

In [ ]:
from carsus.io.nist import NISTWeightsComp, NISTIonizationEnergies
from carsus.io.kurucz import GFALLReader
from carsus.io.zeta import KnoxLongZeta
from carsus.io.chianti_ import ChiantiReader
from carsus.io.cmfgen import CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser, CMFGENReader
from carsus.io.nuclear import NNDCReader
from carsus.io.output import TARDISAtomData

In [ ]:
GFALL_IONS = "H-Si"
CHIANTI_IONS = "H-He"
CMFGEN_IONS = "Si_I-II"

In [ ]:
fname = f"test_data_ku_{GFALL_IONS}_ch_{CHIANTI_IONS}_cm_{CMFGEN_IONS}.h5"
refdata = pd.HDFStore(fname)

In [ ]:
atomic_weights = NISTWeightsComp()
ionization_energies = NISTIonizationEnergies(GFALL_IONS)
gfall_reader = GFALLReader(ions=GFALL_IONS)
chianti_reader = ChiantiReader(ions=CHIANTI_IONS, collisions=True, priority=20)
zeta_data = KnoxLongZeta()
nndc_reader = NNDCReader()

In [ ]:
si_0_lvl = CMFGENEnergyLevelsParser('./cmfgen/energy_levels/SiI_OSC')
si_0_osc = CMFGENOscillatorStrengthsParser('./cmfgen/energy_levels/SiI_OSC')
si_1_lvl = CMFGENEnergyLevelsParser('./cmfgen/energy_levels/si2_osc_kurucz')
si_1_osc = CMFGENOscillatorStrengthsParser('./cmfgen/energy_levels/si2_osc_kurucz')

cmfgen_data = {(14,0): {'levels': si_0_lvl.base, 'lines': si_0_osc.base},
               (14,1): {'levels': si_1_lvl.base, 'lines': si_1_osc.base},}

cmfgen_reader = CMFGENReader(cmfgen_data, priority=20)

In [ ]:
atom_data = TARDISAtomData(atomic_weights,
                           ionization_energies,
                           gfall_reader,
                           zeta_data,
                           chianti_reader,
                           cmfgen_reader,
                           nndc_reader)

In [ ]:
atomic_weights = atom_data.atomic_weights.base.loc[1:14]  # H-Si  to do: store intermediate results
ionization_energies = atom_data.ionization_energies.base
ionization_energies_prepared = atom_data.ionization_energies_prepared
levels_all = atom_data._get_all_levels_data().drop(columns=["ds_id"])
levels = atom_data.levels.drop(columns=["ds_id"])
levels_prepared = atom_data.levels_prepared
lines_all = atom_data._get_all_lines_data().drop(columns=["ds_id"])
lines = atom_data.lines.drop(columns=["ds_id"])
lines_prepared = atom_data.lines_prepared
macro_atom = atom_data.macro_atom
macro_atom_prepared = atom_data.macro_atom_prepared
macro_atom_references = atom_data.macro_atom_references
macro_atom_references_prepared = atom_data.macro_atom_references_prepared
collisions = atom_data.collisions.drop(columns=["btemp", "bscups"])
collisions_prepared = atom_data.collisions_prepared
zeta_data = atom_data.zeta_data.base  # to do: store intermediate results
decay_data = atom_data.nndc_reader.decay_data

In [ ]:
refdata.put('atomic_weights', atomic_weights)
refdata.put('ionization_energies', ionization_energies)
refdata.put('ionization_energies_prepared', ionization_energies_prepared)
refdata.put('levels_all', levels_all)
refdata.put('levels', levels)
refdata.put('levels_prepared', levels_prepared)
refdata.put('lines_all', lines_all)
refdata.put('lines', lines)
refdata.put('lines_prepared', lines_prepared)
refdata.put('macro_atom', macro_atom)
refdata.put('macro_atom_prepared', macro_atom_prepared)
refdata.put('macro_atom_references', macro_atom_references)
refdata.put('macro_atom_references_prepared', macro_atom_references_prepared)
refdata.put('collisions', collisions)
refdata.put('collisions_prepared', collisions_prepared)
refdata.put('zeta_data', zeta_data)
refdata.put('nuclear_decay_rad', decay_data)

In [ ]:
refdata.close()